https://www.youtube.com/watch?v=06TE_U21FK4

GET THE CODE FROM THE VIDEO:
https://github.com/nicknochnack/MediaPipePoseEstimation

# 0. Install and Import Dependencies

In [ ]:
!pip install mediapipe opencv-python

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
# VIDEO FEED
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# 1. Make Detections

In [ ]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render detections(骨架)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [ ]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

和上面的這行landmark的loop有關

 Extract landmarks

        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
            

In [ ]:
landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

In [ ]:
landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]

In [ ]:
landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

# 3. Calculate Angles

Leading leg 的關鍵點是 右側：24, 26, 28/ 左側：23, 25,27

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [ ]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

adians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])這行

數據是前面的 cell landmarks[mp_pose.PoseLandmark.關鍵點名稱.value] 所跑出的 x 座標

In [ ]:
hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

In [ ]:
hip, knee, ankle

In [ ]:
calculate_angle(hip, knee, ankle)

In [ ]:
tuple(np.multiply(knee, [640, 480]).astype(int))

In [ ]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Extract landmarks for the left leg
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # Extract landmarks for the right leg
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            # Calculate angle for the left leg
            angle_left = calculate_angle(left_hip, left_knee, left_ankle)

            # Calculate angle for the right leg
            angle_right = calculate_angle(right_hip, right_knee, right_ankle)

            # Visualize angle for the left leg
            cv2.putText(image, f"Left Leg Angle: {angle_left}", tuple(np.multiply(left_knee, [1400, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (245,66,230), 4, cv2.LINE_AA)

            # Visualize angle for the right leg
            cv2.putText(image, f"Right Leg Angle: {angle_right}", tuple(np.multiply(right_knee, [480, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA)

            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(knee, [640,480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA
                                )
            #print(landmarks)           
            
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# 4. Curl Counter

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Function to calculate angle
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(np.degrees(radians))
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

# Knee counter variables
left_counter = 0
right_counter = 0
left_stage = None
right_stage = None

## Setup mediapipe instance
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break  # Break out of the loop if no frame is read

    # Recolor image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Make detection
    results = pose.process(image)

    # Recolor back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    try:
        landmarks = results.pose_landmarks.landmark

        # Extract landmarks for the left leg
        left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

        # Extract landmarks for the right leg
        right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

        # Calculate angle for the left leg
        angle_left = calculate_angle(left_hip, left_knee, left_ankle)

        # Calculate angle for the right leg
        angle_right = calculate_angle(right_hip, right_knee, right_ankle)

        # Visualize angle for the left leg
        cv2.putText(image, f"Left Leg Angle: {angle_left}", tuple(np.multiply(left_knee, [1400, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (245,66,230), 2, cv2.LINE_AA)

        # Visualize angle for the right leg
        cv2.putText(image, f"Right Leg Angle: {angle_right}", tuple(np.multiply(right_knee, [480, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

        # Check the left leg angle 
        if angle_left is not None and angle_left > 170:
            left_stage = "down"
        if angle_left is not None and angle_left < 90 and left_stage == 'down':
            left_stage = "up"
            left_counter += 1
            print(left_counter)

        # Check the right leg angle 
        if angle_right is not None and angle_right > 170:
            right_stage = "down"
        if angle_right is not None and angle_right < 90 and right_stage == 'down':
            right_stage = "up"
            right_counter += 1
            print(right_counter)

    except:
        pass

    # Render knee counter
    # Setup status box
    cv2.rectangle(image, (0, 0), (460, 100), (245, 117, 16), -1)

    # Rep data for left leg
    cv2.putText(image, 'REPS Right', (15, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(right_counter),
                (15, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2, cv2.LINE_AA)

    # Stage data for left leg
    cv2.putText(image, 'RIGHT STAGE', (125, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, right_stage,
                (125, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Rep data for right leg
    cv2.putText(image, 'REPS Left', (250, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(left_counter),
                (250, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (245,66,230), 2, cv2.LINE_AA)

    # Stage data for left leg
    cv2.putText(image, 'LEFT STAGE', (350, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, left_stage,
                (350, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Render detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

    cv2.imshow('Mediapipe Feed', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Output final counters
print(f"Final Left leg Reps: {left_counter}")
print(f"Final Right leg Reps: {right_counter}")

## 影片偵測

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Function to calculate angle
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(np.degrees(radians))
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Video capture from file
cap = cv2.VideoCapture('pathway to tour video/3.mov')

# Knee counter variables
left_counter = 0
right_counter = 0
left_stage = None
right_stage = None

## Setup mediapipe instance
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break  # Break out of the loop if no frame is read

    # Recolor image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Make detection
    results = pose.process(image)

    # Recolor back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    try:
        landmarks = results.pose_landmarks.landmark

        # Extract landmarks for the left leg
        left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

        # Extract landmarks for the right leg
        right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

        # Calculate angle for the left leg
        angle_left = calculate_angle(left_hip, left_knee, left_ankle)

        # Calculate angle for the right leg
        angle_right = calculate_angle(right_hip, right_knee, right_ankle)

        # Visualize angle for the left leg
        cv2.putText(image, f"Left Leg Angle: {angle_left}", tuple(np.multiply(left_knee, [1400, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (245,66,230), 2, cv2.LINE_AA)

        # Visualize angle for the right leg
        cv2.putText(image, f"Right Leg Angle: {angle_right}", tuple(np.multiply(right_knee, [480, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

        # Check the left leg angle 
        if angle_left is not None and angle_left > 170:
            left_stage = "down"
        if angle_left is not None and angle_left < 90 and left_stage == 'down':
            left_stage = "up"
            left_counter += 1
            print(left_counter)

        # Check the right leg angle 
        if angle_right is not None and angle_right > 170:
            right_stage = "down"
        if angle_right is not None and angle_right < 90 and right_stage == 'down':
            right_stage = "up"
            right_counter += 1
            print(right_counter)

    except:
        pass

    # Render knee counter
    # Setup status box
    cv2.rectangle(image, (0, 0), (460, 100), (245, 117, 16), -1)

    # Rep data for left leg
    cv2.putText(image, 'REPS Right', (15, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(right_counter),
                (15, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2, cv2.LINE_AA)

    # Stage data for left leg
    cv2.putText(image, 'RIGHT STAGE', (125, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, right_stage,
                (125, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Rep data for right leg
    cv2.putText(image, 'REPS Left', (250, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(left_counter),
                (250, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (245,66,230), 2, cv2.LINE_AA)

    # Stage data for left leg
    cv2.putText(image, 'LEFT STAGE', (350, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, left_stage,
                (350, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Render detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

    cv2.imshow('Mediapipe Feed', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Output final counters
print(f"Final Left leg Reps: {left_counter}")
print(f"Final Right leg Reps: {right_counter}")